# Programming Assignment 2: Multi-Layer Perceptrons (MLPs)
## Feed-Forward Neural Networks from Scratch

**Objectives:**
- Understand the building blocks of a feed-forward neural network
- Implement a computational graph using only dictionaries and lists (no classes!)
- Implement forward and backward passes for common operations
- Train an MLP on the MNIST handwritten digit dataset
- **(Extra Credit)** Implement a vectorized version for faster training

---
## Background: From Perceptrons to Multi-Layer Perceptrons

### Quick Recap — What We Built in PA1

In PA1, you implemented a **perceptron**: a single neuron that computes a weighted sum of its inputs, adds a bias, and passes the result through an activation function (a step function). The perceptron can learn to classify data that is **linearly separable** — meaning you can draw a straight line (or hyperplane) to separate the classes.

$$y = \text{step}\left(\sum_{i} w_i x_i + b\right)$$

But here's the problem: **most real-world data is not linearly separable.** A single perceptron cannot learn XOR, cannot recognize handwritten digits, and cannot do much of anything interesting. So what do we do?

### Multi-Layer Perceptrons

The answer is beautifully simple: **connect multiple perceptrons together in layers.** This gives us a **Multi-Layer Perceptron (MLP)**, also called a **Feed-Forward Neural Network (FFNN)**.

The word "feed-forward" means data flows in one direction: from input → through hidden layers → to output. There are no loops or cycles.

```
INPUT LAYER          HIDDEN LAYER          OUTPUT LAYER
  (784 neurons)       (e.g. 16 neurons)     (10 neurons)

    x₁ ──────┐
    x₂ ──────┤──→  h₁ ──┐
    x₃ ──────┤──→  h₂ ──┤──→  y₁ (digit 0?)
     ⋮        ├──→  h₃ ──┤──→  y₂ (digit 1?)
     ⋮        ├──→   ⋮   ├──→   ⋮
   x₇₈₄ ────┘──→  h₁₆ ─┘──→  y₁₀ (digit 9?)
```

**Why does stacking layers help?**  Each hidden layer learns to detect increasingly complex **features**:
- **Layer 1** might learn to detect edges and simple strokes
- **Layer 2** might combine edges into curves, loops, and corners
- **Output layer** combines those features to recognize full digits

This is the core idea of **deep learning** — building up complex representations from simple ones.

### Anatomy of a Single Layer

Each layer in an MLP does two things:

**1. Linear Transformation:** Every neuron computes a weighted sum of ALL inputs from the previous layer, plus a bias:

$$z_j = \sum_{i} w_{ji} \cdot a_i + b_j$$

Or in matrix form (which you'll see in the vectorized version):

$$\mathbf{Z} = \mathbf{W} \cdot \mathbf{A} + \mathbf{b}$$

where $\mathbf{W}$ is the weight matrix, $\mathbf{A}$ is the input from the previous layer, and $\mathbf{b}$ is the bias vector.

**2. Non-Linear Activation:** The result is passed through a non-linear function. Without non-linearity, stacking layers would be pointless — a stack of linear transformations is just one big linear transformation! Common activations include:

| Activation | Formula | When to Use |
|-----------|---------|-------------|
| **ReLU** | $\max(0, z)$ | Hidden layers (most common) |
| **Sigmoid** | $\frac{1}{1+e^{-z}}$ | Binary classification output |
| **Softmax** | $\frac{e^{z_i}}{\sum_j e^{z_j}}$ | Multi-class classification output |

### How Does an MLP Learn? The Training Loop

Training an MLP follows a simple recipe repeated thousands of times:

1. **Forward Pass:** Feed an input through the network to get a prediction
2. **Compute Loss:** Measure how wrong the prediction was (using a loss function like cross-entropy)
3. **Backward Pass (Backpropagation):** Compute how much each weight contributed to the error
4. **Update Weights:** Nudge each weight in the direction that reduces the error

The tricky part is step 3 — **backpropagation**. How do we figure out the gradient of the loss with respect to a weight buried deep in the network? The answer is the **chain rule** from calculus, applied systematically through a **computational graph**.

### The Computational Graph: Our Secret Weapon

Here's where things get elegant. Instead of manually deriving gradients for every possible network architecture, we build a **computational graph** — a record of every operation performed during the forward pass. Then we walk backward through this graph, applying the chain rule at each step.

**Example:** Consider computing $L = (a \cdot b + c)^2$

```
Forward:                          Backward (chain rule):
a ─┐                              dL/da = dL/dd · dd/de · de/da
    ├─ multiply → e ─┐                  = 2d    · 1     · b
b ─┘                  │
                      ├─ add → d ─── power(2) → L
c ────────────────────┘
                              dL/dd = 2d
                              dL/de = dL/dd · dd/de = 2d · 1
                              dL/da = dL/de · de/da = 2d · b
                              dL/db = dL/de · de/db = 2d · a
                              dL/dc = dL/dd · dd/dc = 2d · 1
```

Each node in the graph stores:
- Its **value** (computed during the forward pass)
- Its **gradient** (computed during the backward pass)
- A **`_backward` function** that knows how to propagate gradients to its inputs

This is exactly what PyTorch's `autograd` does under the hood! In this assignment, **you will build this system from scratch** using nothing but Python dictionaries.

### What You're Building in This Assignment

Here's the roadmap:

| Part | What You Build | Why It Matters |
|------|---------------|----------------|
| **Part 1** | `create_value` — the Value node | The fundamental building block |
| **Part 2** | Operations (`add`, `multiply`, `relu`, etc.) | Each op builds the graph AND knows its gradient |
| **Part 3** | `softmax`, `initialize_model`, `forward` | Assembling neurons into a full network |
| **Part 4** | `cross_entropy_loss` | Measuring prediction error |
| **Part 5** | `backward` (backpropagation) | Automatically computing ALL gradients |
| **Part 6** | Training loop + MNIST | Putting it all together on real data |
| **Part 7** | Vectorized version (extra credit) | Making it 100-1000x faster with NumPy |

### MNIST: The Dataset

MNIST is the "Hello, World!" of machine learning. It contains 70,000 grayscale images of handwritten digits (0-9), each 28×28 pixels. Your network will take a flattened 784-dimensional vector as input and output probabilities for each of the 10 digit classes.

## Table of Contents

1. [Part 1: Value Nodes and the Computational Graph](#part1)
2. [Part 2: Operations (Forward and Backward)](#part2)
3. [Part 3: Building the MLP](#part3)
4. [Part 4: Loss Functions](#part4)
5. [Part 5: Backpropagation](#part5)
6. [Part 6: Training on MNIST](#part6)
8. [Part 7 (Extra Credit): Vectorized Implementation](#part7)

**Note:** Some parts may be already implemented which will have **(Done for you)** and would be worth **0 points**.

## Setup

Run the cells below to install dependencies and import libraries. You only need to run the `pip install` cell once.

In [ ]:
# Install dependencies (run once and then comment out)
!pip install scikit-learn matplotlib numpy tqdm

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Union
import tqdm as tqdm
# For reproducibility
random.seed(42)
np.random.seed(42)

---
<a id="part1"></a>
## Part 1: Value Nodes and the Computational Graph (0 points)

### The Big Picture

Every number in our neural network — every weight, every bias, every intermediate result — will live inside a **Value node**. A Value node is simply a Python dictionary that wraps a number and carries along some bookkeeping information for automatic differentiation.

Think of it like this: instead of doing `3.0 + 4.0 = 7.0` and forgetting how we got 7, we create Value nodes that *remember* the computation. Later, when we need gradients, we can trace back through the chain of operations.

### What's Inside a Value Node

Each Value node is a dictionary with these keys:

| Key | Type | Purpose |
|-----|------|---------|
| `'data'` | `float` | The actual number this node holds |
| `'grad'` | `float` | The gradient of the loss w.r.t. this value (starts at 0.0) |
| `'_backward'` | `function` | A function that computes gradients for this node's parents |
| `'_prev'` | `list` | The parent Value nodes that were used to create this one |
| `'_op'` | `str` | A label describing the operation ('+', '*', 'ReLU', etc.) |

**Key insight:** The `'grad'` field will eventually hold $\frac{\partial \text{Loss}}{\partial \text{this value}}$ — how much the final loss would change if we slightly changed this value. This is exactly what we need for gradient descent!

### Task 1.1: Implement `create_value` (Done for you)

The `create_value` function has been already implemented for you. But it is important for you to understand how it works as much of the later parts will require the use of values.

**Study this carefully** — every operation you implement later will call `create_value` to create its output node.

In [ ]:
def create_value(data: float, _children: Tuple = (), _op: str = '') -> Dict:
    """
    Create a scalar value node in the computational graph.
    
    Args:
        data: A scalar number
        _children: Tuple of Value nodes that produced this value
        _op: String describing the operation ('+', '*', 'ReLU', etc.)
    
    Returns:
        A dictionary with keys: 'data', 'grad', '_backward', '_prev', '_op'
    """
    return {
        'data': float(data),
        'grad': 0.0,
        '_backward': lambda: None,
        '_prev': list(_children),
        '_op': _op
    }

---
<a id="part2"></a>
## Part 2: Operations — Forward and Backward (15 points)

### How Operations Build the Computational Graph

Now we'll implement the operations that connect Value nodes together. Each operation does **two things**:

1. **Forward computation:** Computes the result (e.g., `a + b = 5`)
2. **Defines `_backward`:** Stores a function that knows how to push gradients backward through this operation

When we later call `backward()` on the final loss, it will walk through the graph in reverse order, calling each node's `_backward` function. This is how gradients automatically flow from the loss all the way back to the first layer's weights.

### The Chain Rule — The Heart of Backpropagation

Every `_backward` function implements the **chain rule**. If an operation computes `out = f(a, b)`, then:

$$\frac{\partial L}{\partial a} += \frac{\partial f}{\partial a} \times \frac{\partial L}{\partial \text{out}}$$

In code, `out['grad']` already contains $\frac{\partial L}{\partial \text{out}}$ (computed by downstream nodes). Our job is to multiply it by the **local derivative** and add it to `a['grad']`.

**Why `+=` instead of `=`?** Because a single Value node might be used in multiple operations. For example, if `a` is used in both `a + b` and `a * c`, gradients from both paths must be summed.

### Visual Intuition

```
          ┌─── out['grad'] is already filled in by downstream operations
          │
    a ───[operation]──→ out
    b ───┘
          │
          └─── _backward() uses out['grad'] and the local derivative
               to compute gradients for a and b
```

### Task 2.1: Implement `add` (2.5 points)

**Addition: `out = a + b`**

The derivative of `a + b` with respect to `a` is 1, and with respect to `b` is also 1. So the gradient passes through unchanged to both inputs:

$$\frac{\partial (a+b)}{\partial a} = 1, \quad \frac{\partial (a+b)}{\partial b} = 1$$

Therefore: `a.grad += 1 * out.grad` and `b.grad += 1 * out.grad`

In [ ]:
def add(a: Dict, b: Dict) -> Dict:
    """
    Addition operation: out = a + b
    
    The gradient flows equally to both inputs (derivative of a+b 
    with respect to both a and b is 1).
    """
    out = create_value(a['data'] + b['data'], (a, b), '+')
    
    def _backward():
        # ============================================================
        # TODO: Implement the backward pass for addition
        # Hint: The gradient of a sum flows equally to both inputs
        # Remember to use += (not =) since a value may be used 
        # in multiple operations
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

### Task 2.2: Implement `multiply` (2.5 points)

**Multiplication: `out = a * b`**

The derivative of `a * b` with respect to `a` is `b`, and with respect to `b` is `a`. This is the "swap rule" — the gradient for one input is the *other* input times the upstream gradient:

$$\frac{\partial (a \cdot b)}{\partial a} = b, \quad \frac{\partial (a \cdot b)}{\partial b} = a$$

Therefore: `a.grad += b.data * out.grad` and `b.grad += a.data * out.grad`

**Intuition:** If `a = 3` and `b = 4`, then `out = 12`. If we increase `a` by a tiny amount $\epsilon$, the output increases by $4\epsilon$ (i.e., the gradient w.r.t. `a` is `b = 4`).

In [ ]:
def multiply(a: Dict, b: Dict) -> Dict:
    """
    Multiplication operation: out = a * b
    
    The gradient with respect to one input is the other input 
    times the upstream gradient.
    """
    out = create_value(a['data'] * b['data'], (a, b), '*')
    
    def _backward():
        # ============================================================
        # TODO: Implement the backward pass for multiplication
        # Hint: d(a*b)/da = b and d(a*b)/db = a
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

### Task 2.3: Implement `power` (2.5 points)

**Power: `out = a^n`** (where `n` is a constant, not a Value node)

From the power rule in calculus:

$$\frac{\partial (a^n)}{\partial a} = n \cdot a^{n-1}$$

Therefore: `a.grad += n * a.data^(n-1) * out.grad`

**Note:** We will use `power` with `n = -1` later to compute division (via `a^(-1) = 1/a`). This is a common trick in computational graphs — building complex operations from simpler primitives.

In [ ]:
def power(a: Dict, n: Union[int, float]) -> Dict:
    """
    Power operation: out = a^n (where n is a constant, not a Value node)
    """
    out = create_value(a['data'] ** n, (a,), f'**{n}')
    
    def _backward():
        # ============================================================
        # TODO: Implement the backward pass for power
        # Hint: d(a^n)/da = n * a^(n-1)
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

### Task 2.4: Implement `relu` (2.5 points)

**ReLU (Rectified Linear Unit): `out = max(0, a)`**

ReLU is the most popular activation function for hidden layers. It's simple but powerful:

$$\text{ReLU}(a) = \begin{cases} a & \text{if } a > 0 \\ 0 & \text{if } a \leq 0 \end{cases}$$

Its derivative is equally simple:

$$\frac{\partial \text{ReLU}(a)}{\partial a} = \begin{cases} 1 & \text{if } a > 0 \\ 0 & \text{if } a \leq 0 \end{cases}$$

**Intuition:** ReLU acts like a gate. When the input is positive, the gradient flows straight through (multiplied by 1). When the input is negative, the gradient is blocked (multiplied by 0). This is what allows neural networks to learn non-linear decision boundaries.

In [ ]:
def relu(a: Dict) -> Dict:
    """
    ReLU activation: out = max(0, a)
    
    Gradient is 1 where input > 0, and 0 otherwise.
    """
    out = create_value(max(0, a['data']), (a,), 'ReLU')
    
    def _backward():
        # ============================================================
        # TODO: Implement the backward pass for ReLU
        # Hint: Gradient passes through where input was positive,
        #       and is blocked where input was negative/zero
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

### Task 2.5: Implement `sigmoid` (2.5 points)

**Sigmoid: `out = σ(a) = 1 / (1 + e^(-a))`**

Sigmoid squashes any input into the range (0, 1), making it useful as an output activation for binary classification (interpreting the output as a probability).

The sigmoid has a beautifully clean derivative:

$$\frac{\partial \sigma(a)}{\partial a} = \sigma(a) \cdot (1 - \sigma(a))$$

Notice that the derivative is expressed in terms of the output itself — so we can compute the gradient using `s` (the sigmoid output) without needing to recompute the exponential.

**Note:** We clip the input to `[-50, 50]` to prevent numerical overflow in `math.exp()`.

In [ ]:
def sigmoid(a: Dict) -> Dict:
    """
    Sigmoid activation: out = 1 / (1 + e^(-a))
    
    Derivative: sigmoid(x) * (1 - sigmoid(x))
    """
    # Clip input for numerical stability
    x = max(min(a['data'], 50), -50)
    s = 1.0 / (1.0 + math.exp(-x))
    out = create_value(s, (a,), 'sigmoid')
    
    def _backward():
        # ============================================================
        # TODO: Implement the backward pass for sigmoid
        # Hint: d(sigmoid)/da = sigmoid * (1 - sigmoid)
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

### Task 2.6: Implement `log_op` and `exp_op` (2.5 points)

We need the natural logarithm and exponential for building our loss function and softmax activation.

**Natural log: `out = ln(a)`**
$$\frac{\partial \ln(a)}{\partial a} = \frac{1}{a}$$

**Exponential: `out = e^a`**
$$\frac{\partial e^a}{\partial a} = e^a$$

(The exponential is its own derivative — one of the most beautiful facts in calculus!)

**Numerical safety:** We use `max(a, 1e-10)` for `log` (to avoid `log(0)`) and clip the input for `exp` (to avoid overflow).

In [ ]:
def log_op(a: Dict) -> Dict:
    """Natural logarithm: out = ln(a)"""
    val = max(a['data'], 1e-10)  # Prevent log(0)
    out = create_value(math.log(val), (a,), 'log')
    
    def _backward():
        # ============================================================
        # TODO: Implement backward for log
        # Hint: d(ln(a))/da = 1/a
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

def exp_op(a: Dict) -> Dict:
    """Exponential: out = e^a"""
    clipped = max(min(a['data'], 88), -88)  # Prevent overflow
    out = create_value(math.exp(clipped), (a,), 'exp')
    
    def _backward():
        # ============================================================
        # TODO: Implement backward for exp
        # Hint: d(e^a)/da = e^a = out.data
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

### Test your operations

Run the cell below to verify your forward passes are correct. If any assertion fails, go back and check your implementation. (These only test the forward pass — backward pass correctness will be tested later when we run backpropagation.)

In [ ]:
# Test add
a = create_value(2.0)
b = create_value(3.0)
c = add(a, b)
assert c['data'] == 5.0, f"add forward failed: expected 5.0, got {c['data']}"

# Test multiply
a = create_value(2.0)
b = create_value(3.0)
c = multiply(a, b)
assert c['data'] == 6.0, f"multiply forward failed: expected 6.0, got {c['data']}"

# Test power
a = create_value(3.0)
c = power(a, 2)
assert c['data'] == 9.0, f"power forward failed: expected 9.0, got {c['data']}"

# Test relu
a = create_value(-3.0)
c = relu(a)
assert c['data'] == 0.0, f"relu forward failed for negative input"
a = create_value(3.0)
c = relu(a)
assert c['data'] == 3.0, f"relu forward failed for positive input"

# Test sigmoid
a = create_value(0.0)
c = sigmoid(a)
assert abs(c['data'] - 0.5) < 1e-6, f"sigmoid(0) should be 0.5, got {c['data']}"

print("✓ All forward passes work correctly!")

---
<a id="part3"></a>
## Part 3: Softmax and Building the MLP (30 points)

Now we assemble our operations into a complete neural network! This part has three components:

1. **Softmax** — converts raw scores (logits) into probabilities
2. **Model initialization** — creates the network's weights and biases
3. **Forward pass** — feeds data through the network

### Task 3.1: Implement `softmax` (0 points) (Done for you)

Softmax converts a list of logits (raw scores) into a probability distribution that sums to 1. It's used as the output activation for **multi-class classification**.

$$\text{softmax}(x_i) = \frac{e^{x_i - \max(x)}}{\sum_j e^{x_j - \max(x)}}$$

**Why subtract the max?** For numerical stability. Without it, `e^x` can overflow for large values. Subtracting the max doesn't change the result (it cancels out) but keeps the numbers manageable.

**Why is this done for you?** The implementation is a great example of building a complex operation from our primitives. Study it carefully — it uses `add`, `multiply`, `exp_op`, and `power` to build softmax entirely within our computational graph, so gradients flow automatically!

**How it works step-by-step:**
1. Find the maximum logit value (for stability)
2. For each logit: compute `exp(logit - max)`
3. Sum all the exponentials (by chaining `add` operations)
4. Divide each exponential by the sum (using `power(sum, -1)` to get `1/sum`, then `multiply`)

In [ ]:
def softmax(logits: List[Dict]) -> List[Dict]:
    """
    Softmax activation applied to a list of Value nodes.
    
    Args:
        logits: List of Value dictionaries (the raw scores)
    
    Returns:
        List of Value dictionaries (probabilities that sum to 1)
    
    Implementation steps:
        1. Find max value (for numerical stability) — no grad needed
        2. Compute exp(x_i - max) for each logit
        3. Sum all exponentials
        4. Divide each exp by the sum (use power(sum, -1) then multiply)
    """
    max_val = max(v['data'] for v in logits)
    max_node = create_value(max_val)
    
    # Exponentials
    exps = []
    for v in logits:
        shifted = add(v, multiply(max_node, create_value(-1.0)))
        exps.append(exp_op(shifted))
    
    # Sum
    sum_node = exps[0]
    for i in range(1, len(exps)):
        sum_node = add(sum_node, exps[i])
    
    # Probabilities
    inv_sum = power(sum_node, -1)
    probs = [multiply(e, inv_sum) for e in exps]
    
    return probs

### Task 3.2: Implement `initialize_model` (5 points)

Now we create the actual neural network structure. Our MLP is stored as a dictionary of layers, where each layer contains a list of neurons, and each neuron has its own weights and bias — all as Value nodes.

**The data structure looks like this:**

```python
model = {
    'layers': [
        {   # Layer 0 (first hidden layer)
            'neurons': [
                {'w': [Value, Value, ...], 'b': Value},  # Neuron 0
                {'w': [Value, Value, ...], 'b': Value},  # Neuron 1
                ...
            ]
        },
        {   # Layer 1 (output layer)
            'neurons': [...]
        }
    ],
    'activations': ['relu', 'softmax']
}
```

**Weight Initialization Matters!** We use **Xavier initialization**: weights are drawn from a Gaussian distribution with standard deviation $\sqrt{1/n_{\text{in}}}$ where $n_{\text{in}}$ is the number of inputs to the layer. This keeps the signal from exploding or vanishing as it passes through layers.

**What you need to implement:** For each layer, create `layer_nout` neurons, where each neuron has:
- `'w'`: a list of `layer_nin` Value nodes, each initialized with `random.gauss(0, scale)`
- `'b'`: a single Value node initialized to `0.0`

In [ ]:
def initialize_model(nin: int, layer_sizes: List[int], activations: List[str] = None) -> Dict:
    """
    Create an MLP as a dictionary of layers.
    
    Args:
        nin: Input dimension (e.g., 784 for MNIST)
        layer_sizes: List of output sizes for each layer (e.g., [16, 10])
        activations: List of activation functions (e.g., ['relu', 'softmax'])
    
    Returns:
        Dictionary with 'layers' and 'activations' keys
    """
    if activations is None:
        activations = ['relu'] * (len(layer_sizes) - 1) + ['linear']
    
    sz = [nin] + layer_sizes
    layers = []
    
    for i in range(len(layer_sizes)):
        layer_nin = sz[i]
        layer_nout = sz[i + 1]
        scale = math.sqrt(1.0 / layer_nin)  # Xavier initialization
        
        # ============================================================
        # TODO: Create a list of neurons for this layer.
        # Each neuron is a dictionary with:
        #   'w': list of layer_nin Value nodes (weights)
        #        initialized with random.gauss(0, scale)
        #   'b': one Value node (bias) initialized to 0.0
        # 
        # Append {'neurons': neurons_list} to layers
        # ============================================================
        pass
    
    return {
        'layers': layers,
        'activations': activations
    }

### Task 3.3: Implement `forward` (25 points)

The forward pass is where data actually flows through the network. For each layer, you'll:

1. **Linear transformation:** For each neuron, compute the weighted sum of inputs plus bias
2. **Activation:** Apply the layer's activation function

**The linear pass for a single neuron:**
```
act = bias
for each (weight, input) pair:
    act = add(act, multiply(weight, input))
```

This computes $z = w_1 x_1 + w_2 x_2 + \cdots + w_n x_n + b$

**Then apply activation** based on `mlp['activations'][i]`:
- `'relu'` → apply `relu()` to each value independently
- `'sigmoid'` → apply `sigmoid()` to each value independently  
- `'softmax'` → apply `softmax()` to the entire list (since softmax depends on ALL values)
- `'linear'` → no activation (pass through)

**Important:** Every operation here (`add`, `multiply`, `relu`, etc.) is building the computational graph! When we later call `backward()`, gradients will flow through all these operations automatically.

**Hint:** The function receives `x_raw` as a list of plain floats. The first thing it does is wrap them in Value nodes with `create_value`. After that, everything should use our graph operations.

In [ ]:
def forward(mlp: Dict, x_raw: List[float]) -> List[Dict]:
    """
    Forward pass through the MLP.
    
    Args:
        mlp: The model dictionary
        x_raw: List of input values (raw floats)
    
    Returns:
        List of output Value nodes
    """
    # Convert raw inputs to Value nodes
    current_x = [create_value(v) for v in x_raw]
    
    for i, layer in enumerate(mlp['layers']):
        next_x = []
        activation = mlp['activations'][i]
        
        # ============================================================
        # TODO: 
        # 1. LINEAR PASS: For each neuron in this layer:
        #    - Start with the bias: act = neuron['b']
        #    - For each (weight, input) pair:
        #        act = add(act, multiply(weight, input))
        #    - Append act to next_x
        #
        # 2. ACTIVATION: Apply the appropriate activation
        #    - 'relu':    next_x = [relu(x) for x in next_x]
        #    - 'sigmoid': next_x = [sigmoid(x) for x in next_x]
        #    - 'softmax': next_x = softmax(next_x)
        #    - 'linear':  no change
        # ============================================================
        pass
        
        current_x = next_x
    
    return current_x

### Task 3.4: Implement `get_parameters` (Done for you)

This helper function collects all trainable parameters (weights and biases) from the model into a flat list. We need this during training to:
1. Zero out all gradients before each backward pass
2. Update all parameters after each backward pass

Study how it traverses the nested structure: layers → neurons → weights + bias.

In [ ]:
def get_parameters(mlp: Dict) -> List[Dict]:
    """
    Get all parameter Value nodes from the model.
    
    Returns a flat list of all weight and bias Value nodes.
    """
    params = []
    for layer in mlp['layers']:
        for n in layer['neurons']:
            params.extend(n['w'])
            params.append(n['b'])
    return params

### Test your MLP

Run the cell below to verify that your model initializes correctly and that the forward pass produces valid softmax probabilities (they should sum to 1).

In [ ]:
# Test with a small network
model = initialize_model(3, [4, 2], activations=['relu', 'softmax'])

# Check structure
assert len(model['layers']) == 2, "Should have 2 layers"
assert len(model['layers'][0]['neurons']) == 4, "First layer should have 4 neurons"
assert len(model['layers'][1]['neurons']) == 2, "Second layer should have 2 neurons"
assert len(model['layers'][0]['neurons'][0]['w']) == 3, "First layer neurons should have 3 weights"

# Test forward
x = [1.0, 2.0, 3.0]
out = forward(model, x)
assert len(out) == 2, "Output should have 2 values"

# Check softmax sums to 1
prob_sum = sum(o['data'] for o in out)
assert abs(prob_sum - 1.0) < 1e-6, f"Softmax should sum to 1, got {prob_sum}"

# Check parameters
params = get_parameters(model)
expected_params = 3*4 + 4 + 4*2 + 2  # weights + biases for each layer
assert len(params) == expected_params, f"Expected {expected_params} params, got {len(params)}"

print(f"✓ MLP created successfully! Total parameters: {len(params)}")

---
<a id="part4"></a>
## Part 4: Loss Functions (5 points)

### Why We Need a Loss Function

The loss function measures **how wrong** our network's predictions are. During training, our goal is to minimize this loss. The gradients computed by backpropagation tell us how to adjust each weight to reduce the loss.

### Task 4.1: Implement Cross-Entropy Loss (5 points)

For multi-class classification, we use **cross-entropy loss**:

$$\mathcal{L} = -\log(p_{\text{target}})$$

where $p_{\text{target}}$ is the predicted probability for the **correct** class.

**Intuition:**
- If the model assigns probability 0.9 to the correct class: $L = -\log(0.9) = 0.105$ (low loss — good!)
- If the model assigns probability 0.1 to the correct class: $L = -\log(0.1) = 2.303$ (high loss — bad!)
- If the model assigns probability 0.01 to the correct class: $L = -\log(0.01) = 4.605$ (very high loss — very bad!)

The loss grows rapidly as the predicted probability for the correct class approaches 0, creating a strong gradient signal that pushes the network to fix its mistake.

**Build this using operations you've already defined** (`log_op`, `multiply`, `create_value`). This is key — because the loss is built from our graph operations, gradients flow automatically through the loss into the network!

**Implementation hint:** It's just one line: `multiply(log_op(probs[target_idx]), create_value(-1.0))`

In [ ]:
def cross_entropy_loss(probs: List[Dict], target_idx: int) -> Dict:
    """
    Cross-entropy loss for classification.
    
    Args:
        probs: List of probability Value nodes (output of softmax)
        target_idx: Index of the correct class
    
    Returns:
        loss: A scalar Value node
    """
    # ============================================================
    # TODO: Compute loss = -log(probs[target_idx])
    #
    # Hint: Use log_op() and multiply() with create_value(-1.0)
    # loss = multiply(log_op(probs[target_idx]), create_value(-1.0))
    # ============================================================
    pass

### Test your loss function

In [ ]:
# Test cross-entropy loss
# If model is confident and correct, loss should be low
probs = [create_value(0.9), create_value(0.1)]
loss = cross_entropy_loss(probs, target_idx=0)
assert loss['data'] < 0.2, f"Loss should be low for confident correct prediction, got {loss['data']}"

# If model is confident and wrong, loss should be high
loss_wrong = cross_entropy_loss(probs, target_idx=1)
assert loss_wrong['data'] > 2.0, f"Loss should be high for wrong prediction, got {loss_wrong['data']}"

print(f"✓ Loss for correct prediction: {loss['data']:.4f}")
print(f"✓ Loss for wrong prediction: {loss_wrong['data']:.4f}")

---
<a id="part5"></a>
## Part 5: Backpropagation (0 points)

### Task 5.1: `backward` — Automatic Differentiation (Done for you)

This is where all the magic comes together! The `backward` function performs **automatic differentiation** — it computes the gradient of the loss with respect to **every single value** in the computational graph, all in one pass.

**How it works, step by step:**

**Step 1: Topological Sort.** We need to process nodes in the right order — a node's gradient can only be computed after all its *downstream* nodes have been processed. Topological sort gives us this ordering using depth-first search (DFS).

**Step 2: Seed the gradient.** We set `loss['grad'] = 1.0` because $\frac{\partial L}{\partial L} = 1$.

**Step 3: Walk backward.** We iterate through the nodes in *reverse* topological order. For each node, we call its `_backward()` function, which pushes gradients to its parent nodes using the chain rule.

```
Forward:   input → hidden → output → loss
                                       ↓ (grad = 1.0)
Backward:  input ← hidden ← output ← loss
           (grads flow backward through _backward functions)
```

**After `backward()` completes**, every parameter's `.grad` field contains $\frac{\partial L}{\partial \text{param}}$ — exactly what we need for gradient descent!

**Study this implementation carefully** — it's the same algorithm that powers PyTorch, TensorFlow, and every other modern deep learning framework.

In [ ]:
def backward(loss: Dict):
    """
    Backpropagation: compute gradients for all nodes in the graph.
    
    Args:
        loss: The scalar loss Value node (output of cross_entropy_loss)
    
    Steps:
        1. Build topological order using DFS
        2. Set loss.grad = 1.0
        3. Call _backward() on each node in reverse topological order
    """
    topo = []
    visited = set()
    def build_topo(v):
        if id(v) not in visited:
            visited.add(id(v))
            for child in v.get('_prev', []):
                build_topo(child)
            topo.append(v)
    build_topo(loss)
    
    loss['grad'] = 1.0
    for node in reversed(topo):
        node['_backward']()

---
<a id="part6"></a>
## Part 6: Training on MNIST (50 points)

Now we put everything together and train a real neural network on real data! This is where you'll see all the pieces — Value nodes, operations, forward pass, loss, and backward pass — working together.

### 6.1: Load and Preprocess MNIST (5 points)

The code below loads the MNIST dataset, normalizes pixel values to [0, 1], shuffles the data, and splits it into train/validation/test sets.

**Data shapes** (note the transposition — features are rows, samples are columns):
- `X_train`: shape `(784, 400)` — 400 training images, each 784 pixels
- `Y_train_oh`: shape `(10, 400)` — one-hot encoded labels

We use a small subset (400 train, 100 val, 100 test) because our scalar implementation processes one operation at a time — it works, but it's slow! The extra credit vectorized version will handle much larger datasets.

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt

print("Loading MNIST...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X_all, Y_all = mnist.data, mnist.target.astype(int)
X_all = X_all / 255.0

# --- SHUFFLING STEP ---
# Create an array of indices from 0 to len(X_all)
np.random.seed(42)
indices = np.random.permutation(len(X_all))
# Reorder both X and Y using these indices
X_all = X_all[indices]
Y_all = Y_all[indices]
# ----------------------

# Define split sizes
num_train = 400
num_val = 100
num_test = 100

# Slicing the data (Transposing X so features are rows: 784 x N)
X_train = X_all[:num_train].T
Y_train = Y_all[:num_train]

X_val = X_all[num_train : num_train + num_val].T
Y_val = Y_all[num_train : num_train + num_val]

X_test = X_all[num_train + num_val : num_train + num_val + num_test].T
Y_test = Y_all[num_train + num_val : num_train + num_val + num_test]

def to_one_hot(labels, num_classes=10):
    one_hot = np.zeros((num_classes, len(labels)))
    for i, label in enumerate(labels):
        one_hot[label, i] = 1.0
    return one_hot

Y_train_oh = to_one_hot(Y_train)
Y_val_oh = to_one_hot(Y_val)
Y_test_oh = to_one_hot(Y_test)

print(f"Shuffled Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Visualization to confirm variety after shuffling
fig, axes = plt.subplots(1, 5, figsize=(12, 3))
for i in range(5):
    axes[i].imshow(X_train[:, i].reshape(28, 28), cmap='gray')
    axes[i].set_title(f"Label: {Y_train[i]}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

### 6.2: Implement the `get_accuracy` function (5 points)

This function evaluates how well the model is doing. For each sample:
1. Extract the input (a column of `X`) and convert to a Python list
2. Get the true label (using `np.argmax` on the one-hot encoded `Y`)
3. Run the forward pass to get output probabilities
4. The predicted class is the one with highest probability (`np.argmax`)
5. Count how many predictions match the true labels

**Note:** This is an evaluation function — we don't need gradients here, just predictions. But since our forward pass always builds a graph, it works fine (we just ignore the gradients).

In [ ]:
def get_accuracy(model: Dict, X: np.ndarray, Y: np.ndarray) -> float:
    """
    Compute classification accuracy.
    
    Args:
        model: The MLP dictionary
        X: Input data, shape (features, num_samples) — columns are samples
        Y: Labels, shape (num_classes, num_samples) — one-hot encoded
    
    Returns:
        Accuracy as a percentage (0-100)
    """
    correct = 0
    num_samples = X.shape[1]
    
    # ============================================================
    # TODO: For each sample:
    #   1. Get the input column: X[:, i].tolist()
    #   2. Get the true label: np.argmax(Y[:, i])
    #   3. Run forward pass
    #   4. Get predicted class: np.argmax of output data values
    #   5. Count correct predictions
    #
    # Return (correct / num_samples) * 100
    # ============================================================
    pass

### 6.3: Implement the Training Loop (20 points)

This is the core of machine learning — the training loop that iteratively improves the model. Each iteration processes one training sample and performs these five steps:

**1. Zero Gradients:** Before computing new gradients, reset all parameter gradients to 0. (Otherwise, gradients would accumulate from previous iterations!)

```python
for p in params:
    p['grad'] = 0.0
```

**2. Forward Pass:** Feed the input through the network to get predictions.

```python
outputs = forward(model, x)
```

**3. Compute Loss:** Measure how wrong the prediction was.

```python
loss = cross_entropy_loss(outputs, target)
```

**4. Backward Pass:** Compute gradients of the loss w.r.t. all parameters.

```python
backward(loss)
```

**5. Update Parameters (SGD):** Nudge each parameter in the direction that reduces the loss.

```python
for p in params:
    p['data'] -= learning_rate * p['grad']
```

This is **Stochastic Gradient Descent (SGD)** — "stochastic" because we update after each individual sample (rather than computing the average gradient over the entire dataset).

**The learning rate** controls how big each update step is. Too large → the model overshoots and diverges. Too small → training is painfully slow. `0.01` is a reasonable starting point.

**DO NOT CHANGE CODE OUTSIDE OF TODO!**

In [ ]:
def train(model: Dict, X_train: np.ndarray, Y_train: np.ndarray, 
          X_val: np.ndarray,Y_val: np.ndarray,X_test: np.ndarray, Y_test: np.ndarray,
          epochs: int = 5, learning_rate: float = 0.01, 
          print_every: int = 200):
    """
    Train the MLP using stochastic gradient descent (SGD).
    
    Args:
        model: The MLP dictionary
        X_train: Training inputs (784, N)
        Y_train: Training labels (10, N) one-hot
        X_val: Validation inputs
        Y_val: Validation labels
        X_test: Test inputs
        Y_test: Test labels
        epochs: Number of passes through the data
        learning_rate: Step size for parameter updates
        print_every: Print loss every N samples
    """
    params = get_parameters(model)
    num_samples = X_train.shape[1]
    # Initialize history dictionaries
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        
        # Shuffle training data
        indices = list(range(num_samples))
        random.shuffle(indices)
        
        for step, idx in enumerate(indices):
            # Get single sample
            x = X_train[:, idx].tolist()     # Input features
            target = int(np.argmax(Y_train[:, idx]))  # True class

            # progress bar
            pbar = tqdm(enumerate(indices), total=num_samples, desc=f"Epoch {epoch+1}/{epochs}")
            # ============================================================
            # TODO: Training step
            #
            # 1. ZERO GRADIENTS: Set grad to 0.0 for all params
            #    for p in params:
            #        p['grad'] = 0.0
            #
            # 2. FORWARD PASS: 
            #    outputs = forward(model, x)
            #
            # 3. COMPUTE LOSS:
            #    loss = cross_entropy_loss(outputs, target)
            #
            # 4. BACKWARD PASS:
            #    backward(loss)
            #
            # 5. UPDATE PARAMETERS (SGD):
            #    for p in params:
            #        p['data'] -= learning_rate * p['grad']
            # ============================================================
            pass
            
            # Track loss (uncomment after implementing above)
            # epoch_loss += loss['data']
            # if (step + 1) % print_every == 0:
            #     avg_loss = epoch_loss / (step + 1)
            #     print(f"  Epoch {epoch+1}, Step {step+1}/{num_samples}, "
            #           f"Avg Loss: {avg_loss:.4f}")
        
        # Training Metrics
        avg_train_loss = epoch_loss / num_samples
        train_acc = get_accuracy(model, X_train, Y_train)
        
        # Validation Metrics 
        # (Assuming you have a function to calculate total loss over a set)
        val_acc = get_accuracy(model, X_val, Y_val)
        # For val_loss, we'll approximate using the average of a few samples or the full set
        val_loss = get_avg_loss(model, X_val, Y_val) 
        
        # Store in history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        tqdm.write(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Acc: {val_acc:.1f}%")
    
    return history

def get_avg_loss(model, X, Y):
    """Helper to get average loss over a dataset."""
    total_loss = 0
    num_samples = X.shape[1]
    for i in range(num_samples):
        x = X[:, i].tolist()
        target = int(np.argmax(Y[:, i]))
        outputs = forward(model, x)
        loss = cross_entropy_loss(outputs, target)
        total_loss += loss['data']
    return total_loss / num_samples

### 6.4: Train the model! (10 points)

Now let's actually train! We create a network with architecture `784 → 16 → 10`:
- **784** input neurons (one per pixel)
- **16** hidden neurons with ReLU activation
- **10** output neurons with softmax activation (one per digit class)

**What to expect:**
- The scalar implementation processes ~5-20 samples/second
- Each epoch takes ~2 minutes on 400 samples
- 5 epochs total ≈ ~10 minutes
- You should see test accuracy above **75%** when training finishes

**Patience is a virtue here!** The scalar approach is intentionally slow so you can see every operation happening. The vectorized version (Part 7) will do this in under 5 seconds.

In [ ]:
# Create a small MLP: 784 -> 16 -> 10
# Using a small hidden layer because scalar operations are slow
model = initialize_model(784, [16, 10], activations=['relu', 'softmax'])
params = get_parameters(model)
print(f"Model: 784 -> 16 -> 10")
print(f"Total parameters: {len(params)}")
print(f"Training on {X_train.shape[1]} samples...\n")

# Train! (this will take a few minutes)
loss_history = train(
    model, X_train, Y_train_oh, X_test, X_val, Y_val_oh, Y_test_oh,
    epochs=5,
    learning_rate=0.01,
    print_every=200
)

### Plot Training Results

The plots below show how loss and accuracy evolve during training. You should see:
- **Loss decreasing** over epochs (the model is getting better at predicting)
- **Accuracy increasing** over epochs
- If validation loss starts increasing while training loss decreases, that's **overfitting** — the model is memorizing the training data rather than learning general patterns

In [ ]:
def plot_training_results(history):
    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(epochs, history['train_loss'], 'b-o', label='Train Loss')
    ax1.plot(epochs, history['val_loss'], 'r-o', label='Val Loss')
    ax1.set_title('Training & Validation Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    ax2.plot(epochs, history['train_acc'], 'b-o', label='Train Acc')
    ax2.plot(epochs, history['val_acc'], 'r-o', label='Val Acc')
    ax2.set_title('Training & Validation Accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

plot_training_results(loss_history)

final_test_acc = get_accuracy(model, X_test, Y_test_oh)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")

### Visualize Predictions

Let's look at some actual predictions! Green titles mean the model got it right, red means it got it wrong. Even with a tiny network and limited data, you should see many correct predictions.

In [ ]:
# Visualize some predictions
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
for i in range(10):
    ax = axes[i // 5][i % 5]
    idx = i
    x = X_test[:, idx].tolist()
    true_label = int(np.argmax(Y_test_oh[:, idx]))
    
    outputs = forward(model, x)
    pred_label = int(np.argmax([o['data'] for o in outputs]))
    confidence = max(o['data'] for o in outputs)
    
    ax.imshow(X_test[:, idx].reshape(28, 28), cmap='gray')
    color = 'green' if pred_label == true_label else 'red'
    ax.set_title(f"Pred: {pred_label} (True: {true_label})", color=color)
    ax.axis('off')
plt.suptitle("Model Predictions (Green=Correct, Red=Wrong)")
plt.tight_layout()
plt.show()

### 6.5: Improve the model (10 points)

Now it's your turn to experiment. Can you beat the baseline accuracy?

**Hyperparameters to try tuning:**
- **Learning rate:** Try values like 0.001, 0.005, 0.01, 0.05, 0.1. Larger learning rates train faster but may overshoot; smaller ones are more stable but slower.
- **Number of epochs:** More epochs = more passes through the data = potentially better accuracy (but watch for overfitting!)
- **Training size:** More data generally helps. Try increasing `num_train`.

**Architecture changes:**
- **More hidden units:** Try `[32, 10]` or `[64, 10]` instead of `[16, 10]` (but beware: larger networks take longer to train with our scalar implementation)
- **More layers:** Try `[32, 16, 10]` for a 3-layer network. More layers can learn more complex features, but are harder to train.
- **Different activations:** Try `sigmoid` instead of `relu` for hidden layers. You could also experiment with `linear` (though this usually performs worse because it removes the non-linearity that makes deep networks powerful).

**Do not change the output layer activation** — it must remain `softmax` for multi-class classification to work correctly.

**Target:** Your custom model should achieve around **~82%** test accuracy for full credit. **Plot the training and accuracy curves** for your improved model. **Print out the test accuracy and accuracy plots and keep them displayed when submitting.**

**Note:** A larger model may take significantly longer to train. Plan accordingly!

In [ ]:
# TODO Your code goes here

---
<a id="part7"></a>
## Part 7 (Extra Credit): Vectorized Implementation (20 points)

### Why Vectorize?

Our scalar implementation processes **one number at a time** — each weight, each multiplication, each addition is a separate operation. For a network with 12,000+ parameters processing 400 samples, that's millions of individual Python operations per epoch. Python is slow at this!

A **vectorized** implementation uses NumPy to process **entire layers at once** using matrix operations. Instead of 784 individual multiply-and-add operations per neuron, we do a single matrix multiplication:

```
Scalar (slow):                    Vectorized (fast):
for each neuron:                  Z = W @ A + b      ← one operation!
    for each input:               A = relu(Z)        ← one operation!
        act += w[i] * x[i]
    act = relu(act)
```

NumPy's matrix operations run in optimized C/Fortran code and can leverage CPU vectorization (SIMD instructions). This makes the vectorized version **100-1000x faster**.

### What Changes?

| Aspect | Scalar Version | Vectorized Version |
|--------|---------------|-------------------|
| `data` type | `float` | `np.ndarray` |
| Weights | List of Value scalars | Single Value matrix |
| Forward pass | Loop over neurons & inputs | `Z = W @ A + b` |
| Backward pass | Chain rule on scalars | Matrix calculus |
| Batch processing | One sample at a time | All samples at once |

### Key Insight: Matrix Calculus for Backprop

For the linear transformation $Z = W \cdot A + b$:

$$\frac{\partial L}{\partial W} = \frac{\partial L}{\partial Z} \cdot A^T, \quad \frac{\partial L}{\partial A} = W^T \cdot \frac{\partial L}{\partial Z}, \quad \frac{\partial L}{\partial b} = \text{sum over samples of } \frac{\partial L}{\partial Z}$$

This is the matrix version of our scalar chain rule!

### Task 7.1: Implement vectorized `create_value` (3 points)

Same concept as the scalar version, but `data` and `grad` are now numpy arrays instead of floats.

In [ ]:
def create_value_vec(data, _children=None, _op=''):
    """
    Create a vectorized Value node.
    
    Same idea as scalar version, but data and grad are numpy arrays.
    """
    if _children is None:
        _children = ()
    
    # ============================================================
    # TODO: Same as scalar create_value, but:
    #   - Convert data to np.array with dtype=np.float64
    #   - Initialize grad as np.zeros_like(data)
    #   - Add '_id': id(data) for cycle detection in backward
    # ============================================================
    pass

### Task 7.2: Implement vectorized operations (7 points)

The key operations for the vectorized version. The main challenge is handling **broadcasting** in the backward pass — when shapes don't match during addition (like adding a bias vector to a matrix), NumPy broadcasts automatically. But during backprop, we need to sum over the broadcasted dimensions to get the correct gradient shape.

**Matrix multiply (`matmul`):**
- Forward: `out = A @ B`
- Backward: `dA = dout @ B^T`, `dB = A^T @ dout`

**Add with broadcasting:**
- Forward: `out = a + b` (NumPy handles broadcasting)
- Backward: Sum over any dimensions that were broadcasted

**ReLU (element-wise):**
- Forward: `out = np.maximum(0, a)`
- Backward: `da = (out > 0) * dout` (element-wise mask)

In [ ]:
def matmul_vec(a, b):
    """Matrix multiplication: out = a @ b"""
    out = create_value_vec(np.dot(a['data'], b['data']), (a, b), '@')
    
    def _backward():
        # ============================================================
        # TODO: Implement backward for matrix multiply
        # a.grad += dout @ b.T
        # b.grad += a.T @ dout
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

def add_vec(a, b):
    """Addition with broadcasting support."""
    out = create_value_vec(a['data'] + b['data'], (a, b), '+')
    
    def _backward():
        # ============================================================
        # TODO: Implement backward for add
        # Handle broadcasting: sum over dimensions that were broadcasted
        # 
        # For each input (a and b):
        #   1. Start with grad = out['grad']
        #   2. If grad has more dims than the input, sum over extra leading dims
        #   3. If any dim of input is 1, sum over that dim (keepdims=True)
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

def relu_vec(a):
    """ReLU: out = np.maximum(0, a)"""
    out = create_value_vec(np.maximum(0, a['data']), (a,), 'ReLU')
    
    def _backward():
        # ============================================================
        # TODO: Gradient flows where input was positive
        # a['grad'] += (out['data'] > 0) * out['grad']
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

def sigmoid_vec(a):
    """Sigmoid activation for arrays."""
    sigmoid_data = 1 / (1 + np.exp(-np.clip(a['data'], -500, 500)))
    out = create_value_vec(sigmoid_data, (a,), 'sigmoid')
    
    def _backward():
        # ============================================================
        # TODO: a['grad'] += out['data'] * (1 - out['data']) * out['grad']
        # ============================================================
        pass
    
    out['_backward'] = _backward
    return out

def softmax_vec(a):
    """Softmax for arrays (column-wise)."""
    shifted = a['data'] - np.max(a['data'], axis=0, keepdims=True)
    exp_data = np.exp(shifted)
    softmax_data = exp_data / np.sum(exp_data, axis=0, keepdims=True)
    out = create_value_vec(softmax_data, (a,), 'softmax')
    
    def _backward():
        # Simplified: pass gradient through (combined with cross-entropy)
        a['grad'] += out['grad']
    
    out['_backward'] = _backward
    return out

### Task 7.3: Implement vectorized model and training (10 points)

Now we assemble everything into a vectorized model. The key differences from the scalar version:

- **Parameters are matrices:** Instead of each neuron having a list of weight Values, we have one weight *matrix* Value per layer
- **`forward_vec` processes all samples at once:** `Z = W @ A + b` computes the output for every sample in the batch simultaneously
- **Fused softmax + cross-entropy:** The gradient of softmax combined with cross-entropy simplifies beautifully to `(predictions - true_labels) / m`. This avoids computing the complex Jacobian of softmax individually.

In [ ]:
def backward_vec(loss):
    """Backpropagation for vectorized graph."""
    # ============================================================
    # TODO: Same as scalar backward, but use np.ones_like for 
    #       initializing loss gradient
    # ============================================================
    pass

def zero_grad_vec(values):
    """Zero all gradients."""
    for v in values:
        v['grad'] = np.zeros_like(v['data'])

def initialize_model_vec(layer_sizes, activations=None, learning_rate=0.01):
    """
    Create a vectorized MLP.
    
    Parameters are stored as full matrices (not individual scalars).
    
    Args:
        layer_sizes: [input_size, hidden1, hidden2, ..., output_size]
        activations: Activation for each layer transition
        learning_rate: Learning rate
    """
    if activations is None:
        activations = ['relu'] * (len(layer_sizes) - 2) + ['sigmoid']
    
    mlp = {
        'layer_sizes': layer_sizes,
        'activations': activations,
        'learning_rate': learning_rate,
        'num_layers': len(layer_sizes),
        'parameters': []
    }
    
    for i in range(1, len(layer_sizes)):
        fan_in, fan_out = layer_sizes[i-1], layer_sizes[i]
        std_dev = np.sqrt(2.0 / (fan_in + fan_out))
        
        # ============================================================
        # TODO: Create weight matrix W and bias vector b as Value nodes
        #   W = create_value_vec(np.random.randn(fan_out, fan_in) * std_dev)
        #   b = create_value_vec(np.zeros((fan_out, 1)))
        #   Append both to mlp['parameters']
        # ============================================================
        pass
    
    return mlp

def forward_vec(mlp, X):
    """Vectorized forward pass: processes all samples at once!"""
    A = create_value_vec(X)
    cache = [A]
    params = mlp['parameters']
    
    for i in range(mlp['num_layers'] - 1):
        W = params[2*i]
        b = params[2*i + 1]
        
        # ============================================================
        # TODO: Z = W @ A + b, then apply activation
        # ============================================================
        pass
        
        cache.append(A)
    
    return A, cache

def categorical_crossentropy_loss_vec(pred, Y):
    """
    Fused softmax + cross-entropy loss (vectorized).
    
    Uses the mathematical shortcut: gradient of (softmax + CE) w.r.t. 
    logits Z is simply (predictions - true_labels) / m
    """
    Z = pred['_prev'][0]  # Input to softmax
    m = Y.shape[1]
    
    cost = -1/m * np.sum(Y * np.log(pred['data'] + 1e-8))
    loss = create_value_vec(cost, (pred,), 'fused_loss')
    
    def _backward():
        Z['grad'] += (pred['data'] - Y) / m
    
    loss['_backward'] = _backward
    return loss

### Task 7.4: Train the vectorized model

Now we can use a **much bigger** network and **much more** data! The architecture below (`784 → 128 → 64 → 10`) has over 100,000 parameters and trains on 10,000 samples — this would take days with the scalar version but completes in seconds here.

**This should be dramatically faster than the scalar version** — expect the entire training run to finish in under a minute.

In [ ]:
def train_vec(mlp, X_train, Y_train, X_test, Y_test,
              epochs=10, batch_size=32, print_every=1):
    """
    Train vectorized MLP with mini-batch gradient descent.
    """
    params = mlp['parameters']
    lr = mlp['learning_rate']
    num_samples = X_train.shape[1]
    loss_history = []
    
    for epoch in range(epochs):
        # Shuffle
        perm = np.random.permutation(num_samples)
        X_shuffled = X_train[:, perm]
        Y_shuffled = Y_train[:, perm]
        
        epoch_loss = 0.0
        num_batches = num_samples // batch_size
        
        for batch_idx in range(num_batches):
            start = batch_idx * batch_size
            end = start + batch_size
            X_batch = X_shuffled[:, start:end]
            Y_batch = Y_shuffled[:, start:end]
            
            # ============================================================
            # TODO:
            # 1. Zero gradients: zero_grad_vec(params)
            # 2. Forward: pred, cache = forward_vec(mlp, X_batch)
            # 3. Loss: loss = categorical_crossentropy_loss_vec(pred, Y_batch)
            # 4. Backward: backward_vec(loss)
            # 5. Update: for p in params: p['data'] -= lr * p['grad']
            # ============================================================
            pass
            
            # epoch_loss += loss['data']
        
        avg_loss = epoch_loss / max(num_batches, 1)
        loss_history.append(avg_loss)
        
        if (epoch + 1) % print_every == 0:
            # Compute accuracy
            pred_test, _ = forward_vec(mlp, X_test)
            pred_labels = np.argmax(pred_test['data'], axis=0)
            true_labels = np.argmax(Y_test, axis=0)
            acc = np.mean(pred_labels == true_labels) * 100
            print(f"Epoch {epoch+1}/{epochs} — Loss: {avg_loss:.4f}, "
                  f"Test Acc: {acc:.1f}%")
    
    return loss_history

In [ ]:
# Create and train vectorized model
# Now we can use a bigger network and more data!
num_train_vec = 10000
num_test_vec = 2000

X_train_v = X_all[:num_train_vec].T
Y_train_v = to_one_hot(Y_all[:num_train_vec])
X_test_v = X_all[60000:60000+num_test_vec].T
Y_test_v = to_one_hot(Y_all[60000:60000+num_test_vec])

mlp_vec = initialize_model_vec(
    [784, 128, 64, 10], 
    activations=['relu', 'relu', 'softmax'],
    learning_rate=0.1
)

print(f"Vectorized Model: 784 -> 128 -> 64 -> 10")
print(f"Training on {num_train_vec} samples...\n")

loss_history_vec = train_vec(
    mlp_vec, X_train_v, Y_train_v, X_test_v, Y_test_v,
    epochs=20, batch_size=64, print_every=2
)

In [ ]:
# Plot vectorized training
plt.figure(figsize=(8, 4))
plt.plot(loss_history_vec, 'r-o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Vectorized MLP Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

---
## Submission Checklist

Before submitting, make sure:

- [ ] **Part 1:** `create_value` creates proper Value nodes *(provided — just make sure you understand it!)*
- [ ] **Part 2:** All operations (`add`, `multiply`, `power`, `relu`, `sigmoid`, `log_op`, `exp_op`) work correctly — forward AND backward
- [ ] **Part 3:** `initialize_model` and `forward` are implemented correctly
- [ ] **Part 4:** `cross_entropy_loss` computes the correct loss
- [ ] **Part 5:** `backward` correctly computes gradients via backpropagation *(provided)*
- [ ] **Part 6:** 
  - `get_accuracy` correctly computes classification accuracy
  - Training loop is implemented and runs successfully
  - Base model achieves >75% test accuracy
  - Improved model achieves ~82% test accuracy with training curves plotted
- [ ] **(Extra Credit) Part 7:** Vectorized implementation trains successfully

**Grading:**
| Part | Points |
|------|--------|
| Part 1: Value Nodes | 0 (provided) |
| Part 2: Operations (forward + backward) | 15 |
| Part 3: Softmax, Model Init, Forward | 30 |
| Part 4: Cross-Entropy Loss | 5 |
| Part 5: Backpropagation | 0 (provided) |
| Part 6: MNIST Training & Improvement | 50 |
| **Total** | **100** |
| Part 7: Vectorized (Extra Credit) | +20 |

**Remember:** No classes allowed! Only dictionaries and lists.